**Library**

In [36]:
import sys
import os
import yaml
import json
import pandas as pd
import random
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report, accuracy_score

In [2]:
sys.path.append(os.path.abspath(os.path.join('..')))
from src.features.hand_crafted import sent2features, sent2labels, sent2tokens, get_relation_features
from src.models.machine_learning import CRFModel, FlatModelWrapper
from src.data_loader.dataset import convert_label_studio_to_ner_data, prepare_re_data_from_json
from sklearn.ensemble import RandomForestClassifier

**Load Config**

In [3]:
with open('../configs/ml_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

**Load Data**

In [4]:
print("Loading data files...")
TRAIN_PATH = '../data/04_model_input/train_dataset.json'
TEST_PATH = '../data/04_model_input/test_dataset.json'
DEV_PATH = '../data/04_model_input/dev_dataset.json'

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_json = json.load(f)

with open(TEST_PATH, 'r', encoding='utf-8') as f:
    test_json = json.load(f)

with open(DEV_PATH, 'r', encoding='utf-8') as f:
    dev_json = json.load(f)

print(f"-> Train raw docs: {len(train_json)}")
print(f"-> Test raw docs: {len(test_json)}")
print(f"-> Dev raw docs: {len(dev_json)}")

Loading data files...
-> Train raw docs: 797
-> Test raw docs: 114
-> Dev raw docs: 228


In [5]:
print("\nConverting to BIO format...")
train_sents = convert_label_studio_to_ner_data(train_json)
test_sents = convert_label_studio_to_ner_data(test_json)
dev_sents = convert_label_studio_to_ner_data(dev_json)


Converting to BIO format...


Converting with Pyvi: 100%|██████████| 228/228 [00:00<00:00, 807.05it/s]


In [6]:
train_sents[0][:20]

[('Bán', 'O'),
 ('lô', 'B-TYPE'),
 ('đất', 'I-TYPE'),
 ('100m²', 'B-AREA'),
 ('ngay', 'O'),
 ('cạnh', 'O'),
 ('nhà', 'O'),
 ('thi_đấu', 'B-ATTR'),
 ('Thủ_Thừa', 'I-ATTR'),
 (',', 'O'),
 ('view', 'B-ATTR'),
 ('hồ', 'I-ATTR'),
 ('sinh_thái', 'I-ATTR'),
 ('mát_mẻ', 'I-ATTR'),
 (',', 'O'),
 ('giá', 'O'),
 ('mềm', 'O'),
 (',', 'O'),
 ('xây_dựng', 'O'),
 ('ở', 'O')]

**Extracting features**

In [7]:
print("\nExtracting features")
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

print(f"-> X_train shape: {len(X_train)} sentences")
print(f"-> X_test shape: {len(X_test)} sentences")


Extracting features
-> X_train shape: 797 sentences
-> X_test shape: 114 sentences


In [8]:
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{'='*20} EVALUATION: {model_name} {'='*20}")
    
    # Lấy danh sách nhãn thực tế (trừ O để F1-score phản ánh đúng thực thể)
    labels = list(set([l for sent in y_true for l in sent]))
    if 'O' in labels: labels.remove('O')
    labels = sorted(labels)
    
    print(f"Accuracy: {metrics.flat_accuracy_score(y_true, y_pred):.4f}")
    print(metrics.flat_classification_report(
        y_true, y_pred, labels=labels, digits=4
    ))

### **NER**

**CRF (Conditional Random Fields)**

In [9]:
print("Training CRF model")
crf = CRFModel(config.get('crf', {}))
crf.train(X_train, y_train)

Training CRF model


In [10]:
# Predict & Evaluate
y_pred_crf = crf.predict(X_test)
evaluate_model(y_test, y_pred_crf, "CRF")


==================== EVALUATION: CRF ====================
Accuracy: 0.7206
              precision    recall  f1-score   support

      B-AREA     0.6199    0.6136    0.6167       295
      B-ATTR     0.5854    0.4800    0.5275       900
       B-LOC     0.6005    0.6501    0.6243       363
         B-O     0.1515    0.1250    0.1370        40
       B-ORG     0.7500    0.1579    0.2609        38
       B-PER     0.4348    0.2703    0.3333        37
     B-PRICE     0.7803    0.8438    0.8108       160
      B-TYPE     0.6395    0.5528    0.5930       199
      I-AREA     0.7143    0.6229    0.6655       297
      I-ATTR     0.5596    0.5108    0.5341      1296
       I-LOC     0.5957    0.6336    0.6141       393
         I-O     0.1515    0.0806    0.1053        62
       I-ORG     0.7619    0.6957    0.7273        23
       I-PER     0.3750    0.2222    0.2791        27
     I-PRICE     0.7188    0.8789    0.7909       256
      I-TYPE     0.3734    0.3933    0.3831       150

   m

In [11]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# crf.save(os.path.join(OUTPUT_MODEL_DIR, 'crf_model.pkl'))

**SVM (Support Vector Machine)**

In [12]:
print("Training SVM Model")
svm = FlatModelWrapper('svm', config.get('svm', {}))
svm.train(X_train, y_train)

Training SVM Model


In [13]:
# Predict & Evaluate
y_pred_svm = svm.predict(X_test)
evaluate_model(y_test, y_pred_svm, "SVM")


==================== EVALUATION: SVM ====================
Accuracy: 0.7439
              precision    recall  f1-score   support

      B-AREA     0.6278    0.6576    0.6424       295
      B-ATTR     0.5922    0.5422    0.5661       900
       B-LOC     0.6677    0.6198    0.6429       363
         B-O     0.2000    0.1000    0.1333        40
       B-ORG     0.5714    0.1053    0.1778        38
       B-PER     0.4167    0.1351    0.2041        37
     B-PRICE     0.8070    0.8625    0.8338       160
      B-TYPE     0.6465    0.6432    0.6448       199
      I-AREA     0.6704    0.6094    0.6384       297
      I-ATTR     0.6077    0.5355    0.5693      1296
       I-LOC     0.6061    0.6616    0.6326       393
         I-O     0.1176    0.0323    0.0506        62
       I-ORG     0.5714    0.1739    0.2667        23
       I-PER     0.5000    0.3333    0.4000        27
     I-PRICE     0.7419    0.8984    0.8127       256
      I-TYPE     0.4819    0.5333    0.5063       150

   m

In [14]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# svm.save(os.path.join(OUTPUT_MODEL_DIR, 'svm_model.pkl'))

**MaxEnt (Maximum Entropy)**

In [15]:
print("Training MaxEnt Model")
maxent = FlatModelWrapper('maxent', config.get('maxent', {}))
maxent.train(X_train, y_train)

Training MaxEnt Model


C:\Users\LEGION\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [16]:
y_pred_maxent = maxent.predict(X_test)
evaluate_model(y_test, y_pred_maxent, "MaxEnt")


==================== EVALUATION: MaxEnt ====================
Accuracy: 0.7291
              precision    recall  f1-score   support

      B-AREA     0.6242    0.6475    0.6356       295
      B-ATTR     0.6197    0.4744    0.5374       900
       B-LOC     0.5878    0.6639    0.6235       363
         B-O     0.1143    0.1000    0.1067        40
       B-ORG     0.6000    0.0789    0.1395        38
       B-PER     0.5000    0.3514    0.4127        37
     B-PRICE     0.7500    0.8438    0.7941       160
      B-TYPE     0.6519    0.5930    0.6211       199
      I-AREA     0.7160    0.6027    0.6545       297
      I-ATTR     0.6024    0.5085    0.5515      1296
       I-LOC     0.6081    0.6081    0.6081       393
         I-O     0.2273    0.0806    0.1190        62
       I-ORG     0.6667    0.2609    0.3750        23
       I-PER     0.4375    0.2593    0.3256        27
     I-PRICE     0.7324    0.8555    0.7892       256
      I-TYPE     0.4313    0.4600    0.4452       150

 

In [17]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# maxent.save(os.path.join(OUTPUT_MODEL_DIR, 'maxent_model.pkl'))

In [67]:
def visualize_ner_predictions(test_sents, y_true, y_pred, num_samples=2):
    print(f"\n{'='*20} NER PREDICTION SAMPLES {'='*20}")
    
    # Chọn ngẫu nhiên một vài câu để hiển thị
    indices = random.sample(range(len(test_sents)), min(num_samples, len(test_sents)))
    count = 0
    for idx in indices:
        print(f"\n📌 Sentence #{idx}:")
        tokens = [t[0] for t in test_sents[idx]] 
        true_labels = y_true[idx]
        pred_labels = y_pred[idx]

        results = ['✅' if t == p else '❌' for t, p in zip(true_labels, pred_labels)]
        count += sum(1 for r in results if r == '✅')
        print(f"-> Correct Predictions in this sentence: {sum(1 for r in results if r == '✅')} out of {len(results)} tokens.")
        # Tạo DataFrame
        df = pd.DataFrame({
            'Token': tokens,
            'Thực tế (True)': true_labels,
            'Dự đoán (Pred)': pred_labels,
            'Kết quả': results
        })
        display(df)

In [66]:
print("\nVisualizing NER Predictions for CRF Model")
visualize_ner_predictions(test_sents, y_test, y_pred_crf)


Visualizing NER Predictions for CRF Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #65:
-> Correct Predictions in this sentence: 35 out of 58 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Đất,B-TYPE,B-TYPE,✅
1,ngộp,O,O,✅
2,chủ,B-PER,O,❌
3,cần,O,O,✅
4,tiền,O,O,✅
5,gấp,O,O,✅
6,Diện_tích,O,O,✅
7,2084m2,B-AREA,B-AREA,✅
8,ngang,B-AREA,I-AREA,❌
9,20m,I-AREA,I-AREA,✅



📌 Sentence #37:
-> Correct Predictions in this sentence: 47 out of 60 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Nằm,O,O,✅
1,ngay,B-ATTR,O,❌
2,gần,I-ATTR,B-ATTR,❌
3,chợ,I-ATTR,I-ATTR,✅
4,Bà,I-ATTR,I-ATTR,✅
5,Chiểu,I-ATTR,O,❌
6,",",O,O,✅
7,hướng,B-ATTR,O,❌
8,ra,I-ATTR,O,❌
9,Cầu,I-ATTR,B-LOC,❌


In [69]:
print("\nVisualizing NER Predictions for SVM Model")
visualize_ner_predictions(test_sents, y_test, y_pred_svm)


Visualizing NER Predictions for SVM Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #45:
-> Correct Predictions in this sentence: 80 out of 94 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Nguyễn_Thượng_Hiền,B-LOC,B-LOC,✅
1,",",O,O,✅
2,phường,B-LOC,B-LOC,✅
3,Bình_Lợi_Trung,I-LOC,I-LOC,✅
4,",",O,O,✅
...,...,...,...,...
89,có,O,O,✅
90,tâm,O,O,✅
91,và,O,O,✅
92,có,O,O,✅



📌 Sentence #77:
-> Correct Predictions in this sentence: 68 out of 82 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Mở,O,O,✅
1,bán,O,O,✅
2,đất,B-TYPE,B-TYPE,✅
3,nền,I-TYPE,I-TYPE,✅
4,khu,B-LOC,B-LOC,✅
...,...,...,...,...
77,trung_tâm,B-ATTR,B-ATTR,✅
78,thương_mại,I-ATTR,I-ATTR,✅
79,(,O,O,✅
80,Minh_Thiện,B-PER,B-PER,✅


In [68]:
print("\nVisualizing NER Predictions for Maxent Model")
visualize_ner_predictions(test_sents, y_test, y_pred_maxent)


Visualizing NER Predictions for Maxent Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #42:
-> Correct Predictions in this sentence: 49 out of 62 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,"2,5",B-PRICE,B-PRICE,✅
1,tỷ,I-PRICE,I-PRICE,✅
2,Ngay,B-ATTR,O,❌
3,bên,I-ATTR,I-ATTR,✅
4,KCN,I-ATTR,I-ATTR,✅
...,...,...,...,...
57,làm,I-ATTR,O,❌
58,nhà_vườn,I-ATTR,I-ATTR,✅
59,",",O,O,✅
60,xây,B-ATTR,B-ATTR,✅



📌 Sentence #74:
-> Correct Predictions in this sentence: 39 out of 50 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Gia_đình,O,O,✅
1,cần,O,O,✅
2,bán,O,O,✅
3,gấp,O,O,✅
4,2,O,O,✅
5,miếng,O,O,✅
6,đất,B-TYPE,B-TYPE,✅
7,(,O,O,✅
8,5x27m,B-AREA,B-AREA,✅
9,),O,O,✅


### **RE**

In [18]:
print("Creating pairs from Training Data...")
# Lưu ý: train_json, test_json đã được load ở phần NER bên trên
train_pairs = prepare_re_data_from_json(train_json)
test_pairs = prepare_re_data_from_json(test_json)

print(f"-> Total Train Pairs: {len(train_pairs)}")
print(f"-> Total Test Pairs: {len(test_pairs)}")

# 3. Trích xuất đặc trưng & Vector hóa
print("\nExtracting features for Pairs...")

Creating pairs from Training Data...
-> Total Train Pairs: 277036
-> Total Test Pairs: 47950

Extracting features for Pairs...


In [19]:
def extract_re_features(pairs):
    X = []
    y = []
    for p in pairs:
        feats = get_relation_features(p['text'], p['ent1'], p['ent2'])
        X.append(feats)
        y.append(p['label'])
    return X, y

In [20]:
X_train_re_dict, y_train_re = extract_re_features(train_pairs)
X_test_re_dict, y_test_re = extract_re_features(test_pairs)

In [22]:
from sklearn.feature_extraction import DictVectorizer

In [23]:
re_vectorizer = DictVectorizer(sparse=True)
X_train_re = re_vectorizer.fit_transform(X_train_re_dict)
X_test_re = re_vectorizer.transform(X_test_re_dict)

print(f"RE Feature Matrix Shape: {X_train_re.shape}")

RE Feature Matrix Shape: (277036, 92)


In [24]:
# 4. Train Model RE (Random Forest)
print("\nTraining Random Forest for RE")
re_model = RandomForestClassifier(n_estimators=100, random_state=42)
re_model.fit(X_train_re, y_train_re)


Training Random Forest for RE


RandomForestClassifier(random_state=42)

In [ ]:
print("\n--- RE EVALUATION ---")
y_pred_re = re_model.predict(X_test_re)
acc = accuracy_score(y_test_re, y_pred_re)

labels_re = list(re_model.classes_)
if 'NO_RELATION' in labels_re:
    labels_re.remove('NO_RELATION')

print(f"Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print("-" * 30)
print("Relation Extraction Performance:")
print(classification_report(y_test_re, y_pred_re, labels=labels_re))


--- RE EVALUATION ---
-> Overall Accuracy: 0.9760 (97.60%)
------------------------------
Relation Extraction Performance:
              precision    recall  f1-score   support

    HAS_AREA       0.59      0.59      0.59       273
    HAS_ATTR       0.64      0.68      0.66       919
   HAS_PRICE       0.53      0.55      0.54       146
  LOCATED_AT       0.68      0.74      0.71       297

   micro avg       0.63      0.66      0.65      1635
   macro avg       0.61      0.64      0.62      1635
weighted avg       0.63      0.66      0.65      1635



In [71]:
def visualize_re_predictions(test_pairs, y_true, y_pred, num_samples=10):
    print(f"\n{'='*20} RE PREDICTION SAMPLES {'='*20}")
    
    indices = random.sample(range(len(test_pairs)), min(num_samples, len(test_pairs)))
    
    data = []
    for idx in indices:
        pair = test_pairs[idx]
        ent1_text = pair['ent1']['text']
        ent2_text = pair['ent2']['text']
        
        # Format cột đầu tiên: "Sổ đỏ -> Chính chủ"
        pair_str = f"{ent1_text} -> {ent2_text}"
        
        data.append({
            'Cặp thực thể (Entity Pair)': pair_str,
            'Thực tế (True)': y_true[idx],
            'Dự đoán (Pred)': y_pred[idx],
            'Kết quả': '✅' if y_true[idx] == y_pred[idx] else '❌'
        })
    
    df = pd.DataFrame(data)
    
    try:
        from IPython.display import display
        display(df)
    except:
        print(df)

In [94]:
visualize_re_predictions(test_pairs, y_test_re, y_pred_re)


==================== RE PREDICTION SAMPLES ====================


,Cặp thực thể (Entity Pair),Thực tế (True),Dự đoán (Pred),Kết quả
0,Khu vườn thần tiên -> Nhận diện khuân mặt AI,NO_RELATION,NO_RELATION,✅
1,đất -> Cách cao tốc Thành phố Hồ Chí Minh,HAS_ATTR,HAS_ATTR,✅
2,"S1411 -> 4, x tỷ",NO_RELATION,NO_RELATION,✅
3,huyện Bến Cầu -> Huyện Bến Cầu,NO_RELATION,NO_RELATION,✅
4,Đôn Thuận -> Đường nhựa trước đất 6m,NO_RELATION,NO_RELATION,✅
5,căn hộ -> bệnh viện,HAS_ATTR,HAS_ATTR,✅
6,23 tỷ -> Căn Thiên Nga,NO_RELATION,NO_RELATION,✅
7,100m² -> Shophouse đại lộ 51m,NO_RELATION,NO_RELATION,✅
8,Maxhomes -> 98m²,NO_RELATION,NO_RELATION,✅
9,"Sàn giao dịch bất động sản Maxhomes SH4 -> 4,5m",NO_RELATION,NO_RELATION,✅
